# G01: Phase Locking Value (PLV)

**Duration**: 65 minutes  
**Prerequisites**: B01 (Hilbert Transform), B02 (Working with Phase), E02 (Introduction to Hyperscanning)  
**Next**: G02 (Phase Lag Index)

---

## Learning Objectives

By the end of this notebook, you will be able to:
1. Define phase locking value as consistency of phase difference
2. Implement PLV computation from instantaneous phases
3. Interpret PLV values (0 = no locking, 1 = perfect locking)
4. Understand PLV's sensitivity to volume conduction
5. Compute time-resolved PLV for dynamic connectivity
6. Build PLV matrices for multi-channel analysis
7. Apply PLV to hyperscanning inter-brain synchrony

In [ ]:
# Required imports
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import pearsonr
from typing import Tuple, Any

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Color palette
COLORS = {
    'signal_1': '#2E86AB',
    'signal_2': '#E94F37',
    'subject_1': '#2E86AB',
    'subject_2': '#A23B72',
    'highlight': '#F18F01',
    'warning': '#C73E1D'
}

---

## Section 1: Introduction — Are the Phases Locked?

In our previous explorations, we learned about instantaneous phase—the position of a signal within its oscillation cycle at any given moment. Now we face a fundamental question: **Do two signals oscillate "together"?**

**Phase locking** refers to a consistent phase relationship over time. Importantly, this doesn't necessarily mean zero phase difference! Two signals could be 90° apart, but if they're *consistently* 90° apart, they're phase locked. The key is consistency, not identity.

Think of two dancers performing together. They're phase locked if they always maintain the same relative position in their movements—whether perfectly in sync or offset by a fixed amount. They're *not* phase locked if they're sometimes together, sometimes apart, with no consistent pattern.

**Why does PLV matter for neuroscience?** Neural communication is believed to occur through oscillations. When brain regions need to communicate, their oscillations become coherent—aligned in phase—enabling information transfer. PLV captures this functional connectivity by measuring the consistency of phase relationships.

> **Key message**: "PLV measures how consistently two signals maintain their phase relationship."

In [ ]:
# Visualization 1: Phase-locked vs not phase-locked signals

def generate_signals_demo(n_samples: int, fs: float, freq: float, 
                          phase_locked: bool, seed: int = 42) -> Tuple[NDArray, NDArray]:
    """Generate demo signals that are either phase-locked or not."""
    np.random.seed(seed)
    t = np.arange(n_samples) / fs
    
    x = np.sin(2 * np.pi * freq * t)
    
    if phase_locked:
        # Constant phase offset
        phase_offset = np.pi / 4  # 45 degrees
        y = np.sin(2 * np.pi * freq * t + phase_offset)
    else:
        # Variable phase offset (random walk)
        phase_noise = np.cumsum(np.random.randn(n_samples) * 0.1)
        y = np.sin(2 * np.pi * freq * t + phase_noise)
    
    return x, y, t

def extract_phase(sig: NDArray) -> NDArray:
    """Extract instantaneous phase using Hilbert transform."""
    analytic = signal.hilbert(sig)
    return np.angle(analytic)

# Generate both scenarios
fs, freq, duration = 500, 10, 2  # Hz, Hz, seconds
n_samples = int(fs * duration)

x_locked, y_locked, t = generate_signals_demo(n_samples, fs, freq, phase_locked=True)
x_unlocked, y_unlocked, _ = generate_signals_demo(n_samples, fs, freq, phase_locked=False)

# Extract phases and compute differences
phase_x_locked = extract_phase(x_locked)
phase_y_locked = extract_phase(y_locked)
phase_diff_locked = phase_x_locked - phase_y_locked

phase_x_unlocked = extract_phase(x_unlocked)
phase_y_unlocked = extract_phase(y_unlocked)
phase_diff_unlocked = phase_x_unlocked - phase_y_unlocked

# Compute PLV for both
plv_locked = np.abs(np.mean(np.exp(1j * phase_diff_locked)))
plv_unlocked = np.abs(np.mean(np.exp(1j * phase_diff_unlocked)))

# Plot
fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# Left: Phase-locked signals
axes[0, 0].plot(t, x_locked, color=COLORS['signal_1'], label='Signal X', alpha=0.8)
axes[0, 0].plot(t, y_locked, color=COLORS['signal_2'], label='Signal Y', alpha=0.8)
axes[0, 0].set_title(f'Phase-Locked Signals (PLV = {plv_locked:.3f})', fontweight='bold')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].legend()
axes[0, 0].set_xlim([0, 0.5])

axes[1, 0].plot(t, np.mod(phase_diff_locked + np.pi, 2*np.pi) - np.pi, 
                color=COLORS['highlight'], linewidth=1)
axes[1, 0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1, 0].set_title('Phase Difference (Constant)', fontweight='bold')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Δφ (rad)')
axes[1, 0].set_ylim([-np.pi, np.pi])

# Right: Not phase-locked signals
axes[0, 1].plot(t, x_unlocked, color=COLORS['signal_1'], label='Signal X', alpha=0.8)
axes[0, 1].plot(t, y_unlocked, color=COLORS['signal_2'], label='Signal Y', alpha=0.8)
axes[0, 1].set_title(f'Not Phase-Locked Signals (PLV = {plv_unlocked:.3f})', fontweight='bold')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].legend()
axes[0, 1].set_xlim([0, 0.5])

axes[1, 1].plot(t, np.mod(phase_diff_unlocked + np.pi, 2*np.pi) - np.pi, 
                color=COLORS['highlight'], linewidth=1)
axes[1, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1, 1].set_title('Phase Difference (Variable)', fontweight='bold')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Δφ (rad)')
axes[1, 1].set_ylim([-np.pi, np.pi])

plt.tight_layout()
plt.suptitle('Phase Locking: Consistent vs Variable Phase Difference', 
             fontsize=14, fontweight='bold', y=1.02)
plt.show()

---

## Section 2: The Mathematics of PLV

Let's formalize what we mean by phase locking. Given two signals with instantaneous phases φ_x(t) and φ_y(t), we define the **phase difference**:

$$\Delta\phi(t) = \phi_x(t) - \phi_y(t)$$

The clever insight is to represent each phase difference as a **unit vector** in the complex plane:

$$e^{i\Delta\phi(t)}$$

This vector lives on the unit circle—its magnitude is always 1, and its angle equals the phase difference. Now, the **Phase Locking Value** is defined as:

$$PLV = \left| \frac{1}{N} \sum_{t=1}^{N} e^{i(\phi_x(t) - \phi_y(t))} \right|$$

**Interpretation**: We average all the unit vectors over time, then take the magnitude of the result.

- If phase differences are **consistent** (vectors point in similar directions) → large resultant → **high PLV**
- If phase differences are **random** (vectors point everywhere) → vectors cancel → **low PLV**

**Key properties**:
- Range: 0 to 1
- PLV = 1: perfect phase locking (constant Δφ)
- PLV = 0: no phase locking (uniformly random Δφ)
- PLV is **independent of the actual phase difference value**—it measures only consistency

In [ ]:
# Visualization 2: Unit circle diagrams showing PLV as vector resultant

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Generate phase differences for both scenarios
n_points = 50

# Phase-locked: consistent phase difference with small jitter
np.random.seed(42)
phase_diff_clustered = np.pi/4 + np.random.randn(n_points) * 0.2

# Not locked: random phase differences
phase_diff_spread = np.random.uniform(-np.pi, np.pi, n_points)

for ax, phase_diffs, title in [(axes[0], phase_diff_clustered, 'Phase-Locked'),
                                (axes[1], phase_diff_spread, 'Not Phase-Locked')]:
    # Draw unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    ax.plot(np.cos(theta), np.sin(theta), 'k-', alpha=0.3, linewidth=1)
    
    # Plot individual unit vectors as points on circle
    x_points = np.cos(phase_diffs)
    y_points = np.sin(phase_diffs)
    ax.scatter(x_points, y_points, c=COLORS['signal_1'], alpha=0.6, s=30, zorder=3)
    
    # Compute and plot resultant vector
    resultant = np.mean(np.exp(1j * phase_diffs))
    plv = np.abs(resultant)
    
    ax.arrow(0, 0, resultant.real * 0.95, resultant.imag * 0.95,
             head_width=0.08, head_length=0.05, fc=COLORS['highlight'], 
             ec=COLORS['highlight'], linewidth=3, zorder=4)
    
    ax.set_xlim(-1.3, 1.3)
    ax.set_ylim(-1.3, 1.3)
    ax.set_aspect('equal')
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    ax.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
    ax.set_xlabel('Real')
    ax.set_ylabel('Imaginary')
    ax.set_title(f'{title}\nPLV = {plv:.3f}', fontweight='bold', fontsize=12)

plt.suptitle('PLV as Resultant Vector Length on Unit Circle', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Visualization 3: Mathematical diagram showing PLV computation

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Use a small number of vectors for clarity
n_vectors = 8
np.random.seed(123)
phase_diffs = np.pi/3 + np.random.randn(n_vectors) * 0.3  # Clustered around π/3

# Panel 1: Individual unit vectors
ax = axes[0]
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta), np.sin(theta), 'k-', alpha=0.3)

colors = plt.cm.viridis(np.linspace(0.2, 0.8, n_vectors))
for i, pd in enumerate(phase_diffs):
    ax.arrow(0, 0, np.cos(pd)*0.9, np.sin(pd)*0.9, 
             head_width=0.06, head_length=0.04, fc=colors[i], ec=colors[i], linewidth=2)

ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)
ax.set_aspect('equal')
ax.set_title('Step 1: Unit Vectors\n$e^{i\\Delta\\phi(t)}$', fontweight='bold')
ax.set_xlabel('Real')
ax.set_ylabel('Imaginary')

# Panel 2: Sum of vectors (head-to-tail)
ax = axes[1]
ax.plot(np.cos(theta), np.sin(theta), 'k-', alpha=0.3)

# Show head-to-tail addition
cumsum = np.cumsum(np.exp(1j * phase_diffs))
cumsum = np.insert(cumsum, 0, 0)

for i in range(len(phase_diffs)):
    ax.plot([cumsum[i].real, cumsum[i+1].real], 
            [cumsum[i].imag, cumsum[i+1].imag], 
            color=colors[i], linewidth=2, alpha=0.7)

# Final resultant
final = cumsum[-1]
ax.arrow(0, 0, final.real*0.95, final.imag*0.95,
         head_width=0.15, head_length=0.1, fc=COLORS['highlight'], 
         ec=COLORS['highlight'], linewidth=3)

ax.set_xlim(-1.5, max(2, final.real + 1))
ax.set_ylim(-1.5, max(2, final.imag + 1))
ax.set_aspect('equal')
ax.set_title('Step 2: Sum Vectors\n$\\sum e^{i\\Delta\\phi(t)}$', fontweight='bold')
ax.set_xlabel('Real')
ax.set_ylabel('Imaginary')

# Panel 3: Final PLV
ax = axes[2]
plv = np.abs(final) / n_vectors

ax.bar(['PLV'], [plv], color=COLORS['highlight'], edgecolor='black', linewidth=2)
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='Maximum')
ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
ax.set_ylim(0, 1.1)
ax.set_title(f'Step 3: Magnitude / N\nPLV = {plv:.3f}', fontweight='bold')
ax.set_ylabel('PLV Value')

plt.suptitle('PLV Computation Visualized', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 3: Implementing PLV

Now let's implement PLV computation step by step. The pipeline is:

1. **Band-pass filter** both signals (focus on frequency of interest)
2. **Extract instantaneous phase** using Hilbert transform
3. **Compute phase difference**: Δφ(t) = φ_x(t) - φ_y(t)
4. **Convert to unit vectors**: e^(iΔφ(t))
5. **Average across time**
6. **Take magnitude**: PLV = |average|

⚠️ **Critical**: You must filter first! PLV computed on broadband signals is meaningless—the phases of different frequency components would mix chaotically.

In [ ]:
# Function 1: compute_plv

def compute_plv(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    filter_order: int = 4
) -> float:
    """
    Compute Phase Locking Value between two signals.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal (1D array).
    y : NDArray[np.float64]
        Second signal (1D array).
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band of interest (low, high) in Hz.
    filter_order : int, optional
        Order of the Butterworth filter (default: 4).
    
    Returns
    -------
    float
        Phase Locking Value between 0 and 1.
    
    Notes
    -----
    PLV = |mean(exp(i * (phase_x - phase_y)))|
    """
    # Design bandpass filter
    nyq = fs / 2
    low, high = band[0] / nyq, band[1] / nyq
    b, a = signal.butter(filter_order, [low, high], btype='band')
    
    # Filter signals
    x_filt = signal.filtfilt(b, a, x)
    y_filt = signal.filtfilt(b, a, y)
    
    # Extract phases via Hilbert transform
    phase_x = np.angle(signal.hilbert(x_filt))
    phase_y = np.angle(signal.hilbert(y_filt))
    
    # Compute PLV
    phase_diff = phase_x - phase_y
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    
    return float(plv)

In [ ]:
# Function 2: compute_plv_from_phases

def compute_plv_from_phases(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64]
) -> float:
    """
    Compute PLV from pre-computed phases.
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of first signal (radians).
    phase_y : NDArray[np.float64]
        Instantaneous phase of second signal (radians).
    
    Returns
    -------
    float
        Phase Locking Value between 0 and 1.
    
    Notes
    -----
    Use this when phases are already extracted (e.g., for efficiency
    when computing PLV matrix).
    """
    phase_diff = phase_x - phase_y
    plv = np.abs(np.mean(np.exp(1j * phase_diff)))
    return float(plv)

In [ ]:
# Function 3: compute_phase_difference

def compute_phase_difference(
    phase_x: NDArray[np.float64],
    phase_y: NDArray[np.float64],
    wrap: bool = True
) -> NDArray[np.float64]:
    """
    Compute phase difference time series.
    
    Parameters
    ----------
    phase_x : NDArray[np.float64]
        Instantaneous phase of first signal (radians).
    phase_y : NDArray[np.float64]
        Instantaneous phase of second signal (radians).
    wrap : bool, optional
        If True, wrap result to [-π, π] (default: True).
    
    Returns
    -------
    NDArray[np.float64]
        Phase difference time series.
    """
    phase_diff = phase_x - phase_y
    
    if wrap:
        # Wrap to [-π, π]
        phase_diff = np.mod(phase_diff + np.pi, 2 * np.pi) - np.pi
    
    return phase_diff

In [ ]:
# Visualization 4: PLV computation pipeline

# Generate example signals
fs = 500
duration = 2
n_samples = int(fs * duration)
t = np.arange(n_samples) / fs

# Create signals with some phase locking in alpha band
np.random.seed(42)
freq = 10  # Hz (alpha)

# Signal 1: Clean alpha
x = np.sin(2 * np.pi * freq * t) + 0.5 * np.random.randn(n_samples)

# Signal 2: Alpha with consistent phase offset + noise
phase_offset = np.pi / 3
y = np.sin(2 * np.pi * freq * t + phase_offset) + 0.5 * np.random.randn(n_samples)

# Pipeline visualization
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

# Row 1: Raw signals
axes[0, 0].plot(t, x, color=COLORS['signal_1'], alpha=0.8, label='Signal X')
axes[0, 0].plot(t, y, color=COLORS['signal_2'], alpha=0.8, label='Signal Y')
axes[0, 0].set_title('Step 1: Raw Signals', fontweight='bold')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].legend()
axes[0, 0].set_xlim([0, 0.5])

# Row 1b: Filtered signals
band = (8, 12)  # Alpha band
nyq = fs / 2
b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
x_filt = signal.filtfilt(b, a, x)
y_filt = signal.filtfilt(b, a, y)

axes[0, 1].plot(t, x_filt, color=COLORS['signal_1'], alpha=0.8, label='X filtered')
axes[0, 1].plot(t, y_filt, color=COLORS['signal_2'], alpha=0.8, label='Y filtered')
axes[0, 1].set_title('Step 2: Bandpass Filtered (8-12 Hz)', fontweight='bold')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Amplitude')
axes[0, 1].legend()
axes[0, 1].set_xlim([0, 0.5])

# Row 2: Phases
phase_x = np.angle(signal.hilbert(x_filt))
phase_y = np.angle(signal.hilbert(y_filt))

axes[1, 0].plot(t, phase_x, color=COLORS['signal_1'], alpha=0.8, label='Phase X')
axes[1, 0].plot(t, phase_y, color=COLORS['signal_2'], alpha=0.8, label='Phase Y')
axes[1, 0].set_title('Step 3: Extract Phases (Hilbert)', fontweight='bold')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Phase (rad)')
axes[1, 0].legend()
axes[1, 0].set_xlim([0, 0.5])

# Row 2b: Phase difference
phase_diff = compute_phase_difference(phase_x, phase_y)

axes[1, 1].plot(t, phase_diff, color=COLORS['highlight'], alpha=0.8)
axes[1, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[1, 1].set_title('Step 4: Phase Difference', fontweight='bold')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Δφ (rad)')
axes[1, 1].set_ylim([-np.pi, np.pi])

# Row 3: Unit vectors on circle
ax = axes[2, 0]
theta_circle = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta_circle), np.sin(theta_circle), 'k-', alpha=0.3)

# Plot subset of unit vectors
n_show = 100
idx = np.linspace(0, len(phase_diff)-1, n_show, dtype=int)
ax.scatter(np.cos(phase_diff[idx]), np.sin(phase_diff[idx]), 
           c=COLORS['signal_1'], alpha=0.3, s=20)

# Resultant
resultant = np.mean(np.exp(1j * phase_diff))
ax.arrow(0, 0, resultant.real*0.9, resultant.imag*0.9,
         head_width=0.08, head_length=0.05, fc=COLORS['highlight'], 
         ec=COLORS['highlight'], linewidth=3)

ax.set_xlim(-1.3, 1.3)
ax.set_ylim(-1.3, 1.3)
ax.set_aspect('equal')
ax.set_title('Step 5: Unit Vectors & Average', fontweight='bold')
ax.set_xlabel('Real')
ax.set_ylabel('Imaginary')

# Row 3b: Final PLV
plv = np.abs(resultant)
ax = axes[2, 1]
ax.bar(['PLV'], [plv], color=COLORS['highlight'], edgecolor='black', linewidth=2, width=0.5)
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5)
ax.set_ylim(0, 1.1)
ax.set_title(f'Step 6: Take Magnitude\nPLV = {plv:.3f}', fontweight='bold')
ax.set_ylabel('PLV Value')

plt.suptitle('PLV Computation Pipeline', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 4: PLV with Controlled Examples

Let's build intuition by exploring PLV with signals where we control the phase relationship.

In [ ]:
# Function 4: generate_phase_locked_signals

def generate_phase_locked_signals(
    n_samples: int,
    fs: float,
    frequency: float,
    phase_offset: float = 0.0,
    plv_target: float = 1.0,
    seed: int | None = None
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Generate two signals with specified phase locking.
    
    Parameters
    ----------
    n_samples : int
        Number of samples.
    fs : float
        Sampling frequency in Hz.
    frequency : float
        Oscillation frequency in Hz.
    phase_offset : float, optional
        Constant phase offset in radians (default: 0.0).
    plv_target : float, optional
        Target PLV value between 0 and 1 (default: 1.0).
        Values < 1 add phase noise to reduce PLV.
    seed : int | None, optional
        Random seed for reproducibility.
    
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        Two signals (x, y) with the specified phase relationship.
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.arange(n_samples) / fs
    
    # Base phase
    base_phase = 2 * np.pi * frequency * t
    
    # Signal X
    x = np.sin(base_phase)
    
    # Signal Y with offset
    # Add phase noise to reduce PLV below target
    if plv_target < 1.0:
        # Phase noise standard deviation (empirically determined)
        # PLV ≈ exp(-σ²/2) for wrapped normal distribution
        sigma = np.sqrt(-2 * np.log(max(plv_target, 0.01)))
        phase_noise = np.random.randn(n_samples) * sigma
    else:
        phase_noise = 0
    
    y = np.sin(base_phase + phase_offset + phase_noise)
    
    return x, y

In [ ]:
# Visualization 5: Grid of PLV examples

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

fs = 500
n_samples = 5000
freq = 10
band = (8, 12)
t = np.arange(n_samples) / fs

examples = [
    {'phase_offset': 0.0, 'plv_target': 1.0, 'title': 'Identical (Δφ=0, PLV≈1)'},
    {'phase_offset': np.pi/2, 'plv_target': 1.0, 'title': '90° Offset (PLV≈1)'},
    {'phase_offset': np.pi/4, 'plv_target': 0.7, 'title': 'Partial Lock (PLV≈0.7)'},
    {'phase_offset': 0.0, 'plv_target': 0.5, 'title': 'Weak Lock (PLV≈0.5)'},
    {'phase_offset': 0.0, 'plv_target': 0.2, 'title': 'Very Weak (PLV≈0.2)'},
    {'phase_offset': 0.0, 'plv_target': 0.05, 'title': 'No Lock (PLV≈0)'},
]

for ax, ex in zip(axes.flat, examples):
    x, y = generate_phase_locked_signals(
        n_samples, fs, freq, 
        phase_offset=ex['phase_offset'],
        plv_target=ex['plv_target'],
        seed=42
    )
    
    # Compute actual PLV
    plv = compute_plv(x, y, fs, band)
    
    # Plot signals (short segment)
    show_samples = int(0.5 * fs)
    ax.plot(t[:show_samples], x[:show_samples], color=COLORS['signal_1'], 
            alpha=0.8, label='X')
    ax.plot(t[:show_samples], y[:show_samples], color=COLORS['signal_2'], 
            alpha=0.8, label='Y')
    
    ax.set_title(f"{ex['title']}\nMeasured PLV = {plv:.3f}", fontweight='bold')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.legend(loc='upper right', fontsize=8)

plt.suptitle('PLV for Different Phase Relationships', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Visualization 6: PLV vs noise level

plv_targets = np.linspace(0.05, 1.0, 20)
measured_plvs = []

for target in plv_targets:
    x, y = generate_phase_locked_signals(
        n_samples=10000, fs=500, frequency=10,
        plv_target=target, seed=42
    )
    plv = compute_plv(x, y, fs=500, band=(8, 12))
    measured_plvs.append(plv)

plt.figure(figsize=(10, 5))
plt.plot(plv_targets, measured_plvs, 'o-', color=COLORS['signal_1'], 
         linewidth=2, markersize=8)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Ideal')
plt.xlabel('Target PLV (controlled by phase noise)', fontsize=12)
plt.ylabel('Measured PLV', fontsize=12)
plt.title('PLV Decreases with Phase Noise', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 1.05)
plt.ylim(0, 1.05)
plt.show()

---

## Section 5: PLV and Volume Conduction

⚠️ **Critical Limitation**: PLV is sensitive to **volume conduction**.

**The problem**: When electrical activity from a single brain source spreads through the conductive tissue to multiple electrodes, those electrodes record essentially the same signal. This creates:
- Phase difference ≈ 0 (or π)
- Extremely consistent phase difference
- PLV → very high!

But this high PLV is **spurious**—it doesn't reflect true neural connectivity, just the physics of electrical conduction.

**Why PLV fails**: PLV measures consistency of *any* phase difference. It doesn't distinguish:
- Δφ = 0 (suspicious—likely volume conduction)
- Δφ = 30° (more believable—suggests actual signal transmission with some delay)

**Solutions** (previewed):
- **PLI** (G02): Ignores zero-phase contributions
- **wPLI** (G03): Weights by imaginary component

**Good news for hyperscanning**: Between-brain PLV is safe! There's no volume conduction between two separate heads. Within-brain PLV should be interpreted more cautiously.

In [ ]:
# Visualization 7 & 8: Volume conduction demonstration

np.random.seed(42)
fs = 500
n_samples = 5000
t = np.arange(n_samples) / fs
freq = 10
band = (8, 12)

# Source signal
source = np.sin(2 * np.pi * freq * t)

# Volume conduction: same signal + small independent noise at two electrodes
noise_level = 0.1
electrode_1_vc = source + noise_level * np.random.randn(n_samples)
electrode_2_vc = source + noise_level * np.random.randn(n_samples)

# True connectivity: signal with consistent delay
delay_samples = int(0.01 * fs)  # 10ms delay
electrode_1_true = source + 0.3 * np.random.randn(n_samples)
electrode_2_true = np.roll(source, delay_samples) + 0.3 * np.random.randn(n_samples)

# Compute PLVs
plv_vc = compute_plv(electrode_1_vc, electrode_2_vc, fs, band)
plv_true = compute_plv(electrode_1_true, electrode_2_true, fs, band)

# Extract phases for histograms
nyq = fs / 2
b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')

e1_vc_filt = signal.filtfilt(b, a, electrode_1_vc)
e2_vc_filt = signal.filtfilt(b, a, electrode_2_vc)
phase_diff_vc = compute_phase_difference(
    np.angle(signal.hilbert(e1_vc_filt)),
    np.angle(signal.hilbert(e2_vc_filt))
)

e1_true_filt = signal.filtfilt(b, a, electrode_1_true)
e2_true_filt = signal.filtfilt(b, a, electrode_2_true)
phase_diff_true = compute_phase_difference(
    np.angle(signal.hilbert(e1_true_filt)),
    np.angle(signal.hilbert(e2_true_filt))
)

# Plot
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# Row 1: Volume conduction
axes[0, 0].plot(t[:500], electrode_1_vc[:500], color=COLORS['signal_1'], label='Electrode 1')
axes[0, 0].plot(t[:500], electrode_2_vc[:500], color=COLORS['signal_2'], label='Electrode 2')
axes[0, 0].set_title('Volume Conduction: Signals', fontweight='bold')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].legend()

axes[0, 1].hist(phase_diff_vc, bins=50, color=COLORS['warning'], alpha=0.7, density=True)
axes[0, 1].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[0, 1].set_title('Phase Difference Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Δφ (rad)')
axes[0, 1].set_xlim(-np.pi, np.pi)

axes[0, 2].bar(['PLV'], [plv_vc], color=COLORS['warning'], edgecolor='black', width=0.5)
axes[0, 2].axhline(y=1, color='gray', linestyle='--', alpha=0.5)
axes[0, 2].set_ylim(0, 1.1)
axes[0, 2].set_title(f'PLV = {plv_vc:.3f} (SPURIOUS!)', fontweight='bold', color=COLORS['warning'])

# Row 2: True connectivity
axes[1, 0].plot(t[:500], electrode_1_true[:500], color=COLORS['signal_1'], label='Electrode 1')
axes[1, 0].plot(t[:500], electrode_2_true[:500], color=COLORS['signal_2'], label='Electrode 2')
axes[1, 0].set_title('True Connectivity: Signals', fontweight='bold')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].legend()

axes[1, 1].hist(phase_diff_true, bins=50, color=COLORS['signal_1'], alpha=0.7, density=True)
axes[1, 1].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[1, 1].set_title('Phase Difference Distribution', fontweight='bold')
axes[1, 1].set_xlabel('Δφ (rad)')
axes[1, 1].set_xlim(-np.pi, np.pi)

axes[1, 2].bar(['PLV'], [plv_true], color=COLORS['signal_1'], edgecolor='black', width=0.5)
axes[1, 2].axhline(y=1, color='gray', linestyle='--', alpha=0.5)
axes[1, 2].set_ylim(0, 1.1)
axes[1, 2].set_title(f'PLV = {plv_true:.3f} (Genuine)', fontweight='bold')

plt.suptitle("PLV Can't Distinguish Volume Conduction from True Connectivity\n(Both have high PLV!)", 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 6: Time-Resolved PLV

Static PLV gives us a single value for the entire recording. But connectivity is **dynamic**—it changes over time! Time-resolved PLV tracks these changes using a sliding window approach:

1. Extract phases for the full signal
2. Compute PLV in short windows
3. Slide the window across time
4. Get PLV time series

**Trade-offs**:
- **Longer windows**: More stable PLV estimates, but poorer time resolution
- **Shorter windows**: Better time resolution, but noisier estimates

Typical overlap is 50-90% to smooth the time series.

In [ ]:
# Function 5: compute_plv_timeseries

def compute_plv_timeseries(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    window_sec: float = 1.0,
    overlap: float = 0.5
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Compute time-resolved PLV using sliding windows.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal.
    y : NDArray[np.float64]
        Second signal.
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    window_sec : float, optional
        Window size in seconds (default: 1.0).
    overlap : float, optional
        Overlap fraction between windows (default: 0.5).
    
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        (time_centers, plv_values) - Time points and corresponding PLV.
    """
    # Filter and extract phases
    nyq = fs / 2
    b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
    x_filt = signal.filtfilt(b, a, x)
    y_filt = signal.filtfilt(b, a, y)
    
    phase_x = np.angle(signal.hilbert(x_filt))
    phase_y = np.angle(signal.hilbert(y_filt))
    phase_diff = phase_x - phase_y
    
    # Window parameters
    window_samples = int(window_sec * fs)
    step_samples = int(window_samples * (1 - overlap))
    
    # Compute PLV in each window
    time_centers = []
    plv_values = []
    
    start = 0
    while start + window_samples <= len(phase_diff):
        window_phase_diff = phase_diff[start:start + window_samples]
        plv = np.abs(np.mean(np.exp(1j * window_phase_diff)))
        
        time_center = (start + window_samples / 2) / fs
        time_centers.append(time_center)
        plv_values.append(plv)
        
        start += step_samples
    
    return np.array(time_centers), np.array(plv_values)

In [ ]:
# Function 6: compute_plv_epochs

def compute_plv_epochs(
    epochs_x: NDArray[np.float64],
    epochs_y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> NDArray[np.float64]:
    """
    Compute PLV for each epoch separately.
    
    Parameters
    ----------
    epochs_x : NDArray[np.float64]
        Epochs from first signal, shape (n_epochs, n_samples).
    epochs_y : NDArray[np.float64]
        Epochs from second signal, shape (n_epochs, n_samples).
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    
    Returns
    -------
    NDArray[np.float64]
        PLV values for each epoch, shape (n_epochs,).
    """
    n_epochs = epochs_x.shape[0]
    plv_values = np.zeros(n_epochs)
    
    for i in range(n_epochs):
        plv_values[i] = compute_plv(epochs_x[i], epochs_y[i], fs, band)
    
    return plv_values

In [ ]:
# Visualization 9: Time-resolved PLV example

# Create signals where PLV changes over time
np.random.seed(42)
fs = 500
duration = 10  # seconds
n_samples = int(fs * duration)
t = np.arange(n_samples) / fs
freq = 10

# First half: phase locked, Second half: not locked
half = n_samples // 2

x = np.sin(2 * np.pi * freq * t)
y = np.zeros(n_samples)

# First half: consistent phase offset
y[:half] = np.sin(2 * np.pi * freq * t[:half] + np.pi/4)

# Second half: random phase relationship
phase_noise = np.cumsum(np.random.randn(half) * 0.3)
y[half:] = np.sin(2 * np.pi * freq * t[half:] + phase_noise)

# Compute time-resolved PLV
band = (8, 12)
time_centers, plv_ts = compute_plv_timeseries(x, y, fs, band, window_sec=1.0, overlap=0.8)

# Extract phases for phase difference plot
nyq = fs / 2
b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
x_filt = signal.filtfilt(b, a, x)
y_filt = signal.filtfilt(b, a, y)
phase_diff = compute_phase_difference(
    np.angle(signal.hilbert(x_filt)),
    np.angle(signal.hilbert(y_filt))
)

# Plot
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Signals
axes[0].plot(t, x, color=COLORS['signal_1'], alpha=0.7, label='Signal X')
axes[0].plot(t, y, color=COLORS['signal_2'], alpha=0.7, label='Signal Y')
axes[0].axvline(x=duration/2, color='gray', linestyle='--', linewidth=2, label='PLV transition')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Signals with Changing Phase Relationship', fontweight='bold')
axes[0].legend(loc='upper right')

# Phase difference
axes[1].plot(t, phase_diff, color=COLORS['highlight'], alpha=0.5, linewidth=0.5)
axes[1].axvline(x=duration/2, color='gray', linestyle='--', linewidth=2)
axes[1].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
axes[1].set_ylabel('Δφ (rad)')
axes[1].set_ylim(-np.pi, np.pi)
axes[1].set_title('Phase Difference Over Time', fontweight='bold')

# Time-resolved PLV
axes[2].plot(time_centers, plv_ts, color=COLORS['signal_1'], linewidth=2)
axes[2].fill_between(time_centers, 0, plv_ts, color=COLORS['signal_1'], alpha=0.3)
axes[2].axvline(x=duration/2, color='gray', linestyle='--', linewidth=2)
axes[2].axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
axes[2].set_xlabel('Time (s)')
axes[2].set_ylabel('PLV')
axes[2].set_ylim(0, 1)
axes[2].set_title('Time-Resolved PLV (1s window, 80% overlap)', fontweight='bold')

# Add annotations
axes[2].annotate('High PLV\n(locked)', xy=(2.5, 0.85), fontsize=11, ha='center')
axes[2].annotate('Low PLV\n(not locked)', xy=(7.5, 0.3), fontsize=11, ha='center')

plt.suptitle('Time-Resolved PLV Reveals Dynamic Connectivity', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Section 7: PLV Matrix

For multi-channel analysis, we compute PLV between all pairs of channels, creating a **PLV connectivity matrix**.

**Properties**:
- **Symmetric**: PLV(X,Y) = PLV(Y,X)
- **Diagonal = 1**: A signal is perfectly locked with itself
- **Values 0 to 1**

**Efficiency tip**: Extract phases for all channels once, then compute pairwise PLV.

In [ ]:
# Function 7: compute_plv_matrix

def compute_plv_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> NDArray[np.float64]:
    """
    Compute PLV matrix for all channel pairs.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Multi-channel data, shape (n_channels, n_samples).
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    
    Returns
    -------
    NDArray[np.float64]
        PLV matrix, shape (n_channels, n_channels).
    """
    n_channels = data.shape[0]
    
    # Filter all channels
    nyq = fs / 2
    b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
    data_filt = signal.filtfilt(b, a, data, axis=1)
    
    # Extract phases for all channels
    phases = np.angle(signal.hilbert(data_filt, axis=1))
    
    # Compute PLV matrix
    plv_matrix = np.ones((n_channels, n_channels))
    
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            plv = compute_plv_from_phases(phases[i], phases[j])
            plv_matrix[i, j] = plv
            plv_matrix[j, i] = plv  # Symmetric
    
    return plv_matrix

In [ ]:
# Function 8: compute_plv_matrix_bands

STANDARD_BANDS = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}

def compute_plv_matrix_bands(
    data: NDArray[np.float64],
    fs: float,
    bands: dict[str, tuple[float, float]] | None = None
) -> dict[str, NDArray[np.float64]]:
    """
    Compute PLV matrices for multiple frequency bands.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        Multi-channel data, shape (n_channels, n_samples).
    fs : float
        Sampling frequency in Hz.
    bands : dict[str, tuple[float, float]] | None, optional
        Dictionary of band names to (low, high) frequencies.
        If None, uses standard bands.
    
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary of band names to PLV matrices.
    """
    if bands is None:
        bands = STANDARD_BANDS
    
    return {name: compute_plv_matrix(data, fs, band) for name, band in bands.items()}

In [ ]:
# Visualization 10 & 11: PLV matrix examples

# Generate synthetic multi-channel data with cluster structure
np.random.seed(42)
fs = 500
n_samples = 10000
n_channels = 8
t = np.arange(n_samples) / fs

# Create two clusters of phase-locked channels
# Cluster 1: channels 0-3
# Cluster 2: channels 4-7

data = np.zeros((n_channels, n_samples))
freq = 10

# Cluster 1: shared base signal
base_1 = np.sin(2 * np.pi * freq * t)
for i in range(4):
    phase_offset = np.random.randn() * 0.1  # Small random offset within cluster
    data[i] = np.sin(2 * np.pi * freq * t + phase_offset) + 0.3 * np.random.randn(n_samples)

# Cluster 2: different base signal
base_2 = np.sin(2 * np.pi * freq * t + np.pi/2)  # 90° offset from cluster 1
for i in range(4, 8):
    phase_offset = np.random.randn() * 0.1
    data[i] = np.sin(2 * np.pi * freq * t + np.pi/2 + phase_offset) + 0.3 * np.random.randn(n_samples)

# Compute PLV matrix
plv_matrix = compute_plv_matrix(data, fs, (8, 12))

# Plot
fig, ax = plt.subplots(figsize=(8, 7))

im = ax.imshow(plv_matrix, cmap='viridis', vmin=0, vmax=1, aspect='equal')
plt.colorbar(im, ax=ax, label='PLV')

# Add grid lines to show clusters
ax.axhline(y=3.5, color='white', linewidth=2)
ax.axvline(x=3.5, color='white', linewidth=2)

# Labels
ax.set_xticks(range(n_channels))
ax.set_yticks(range(n_channels))
ax.set_xticklabels([f'Ch{i+1}' for i in range(n_channels)])
ax.set_yticklabels([f'Ch{i+1}' for i in range(n_channels)])
ax.set_xlabel('Channel')
ax.set_ylabel('Channel')
ax.set_title('PLV Connectivity Matrix (Alpha Band)\nTwo clusters visible', fontweight='bold')

# Add cluster annotations
ax.text(1.5, -0.8, 'Cluster 1', ha='center', fontsize=10, fontweight='bold')
ax.text(5.5, -0.8, 'Cluster 2', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

---

## Section 8: PLV for Hyperscanning

**This is the key application!** In hyperscanning, we record brain activity from multiple people simultaneously. PLV helps us measure inter-brain synchrony—are the participants' brains oscillating together?

**Structure**:
- **Within-P1 PLV**: Connectivity within participant 1's brain
- **Within-P2 PLV**: Connectivity within participant 2's brain  
- **Between-brain PLV**: Connectivity between P1 and P2 (the exciting part!)

**Research findings**:
- Inter-brain PLV increases during cooperation
- Predicts task success and rapport
- Different frequencies may reflect different social processes

**PLV is the most common hyperscanning connectivity metric!**

In [ ]:
# Function 9: compute_plv_hyperscanning

def compute_plv_hyperscanning(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, NDArray[np.float64]]:
    """
    Compute PLV matrices for hyperscanning data.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        Data from participant 1, shape (n_channels_p1, n_samples).
    data_p2 : NDArray[np.float64]
        Data from participant 2, shape (n_channels_p2, n_samples).
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    
    Returns
    -------
    dict[str, NDArray[np.float64]]
        Dictionary with keys:
        - 'within_p1': PLV matrix within participant 1
        - 'within_p2': PLV matrix within participant 2
        - 'between': PLV matrix between participants
        - 'full': Full combined matrix
    """
    n_ch_p1 = data_p1.shape[0]
    n_ch_p2 = data_p2.shape[0]
    
    # Filter and extract phases
    nyq = fs / 2
    b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
    
    data_p1_filt = signal.filtfilt(b, a, data_p1, axis=1)
    data_p2_filt = signal.filtfilt(b, a, data_p2, axis=1)
    
    phases_p1 = np.angle(signal.hilbert(data_p1_filt, axis=1))
    phases_p2 = np.angle(signal.hilbert(data_p2_filt, axis=1))
    
    # Within-P1 matrix
    within_p1 = np.ones((n_ch_p1, n_ch_p1))
    for i in range(n_ch_p1):
        for j in range(i + 1, n_ch_p1):
            plv = compute_plv_from_phases(phases_p1[i], phases_p1[j])
            within_p1[i, j] = plv
            within_p1[j, i] = plv
    
    # Within-P2 matrix
    within_p2 = np.ones((n_ch_p2, n_ch_p2))
    for i in range(n_ch_p2):
        for j in range(i + 1, n_ch_p2):
            plv = compute_plv_from_phases(phases_p2[i], phases_p2[j])
            within_p2[i, j] = plv
            within_p2[j, i] = plv
    
    # Between matrix
    between = np.zeros((n_ch_p1, n_ch_p2))
    for i in range(n_ch_p1):
        for j in range(n_ch_p2):
            between[i, j] = compute_plv_from_phases(phases_p1[i], phases_p2[j])
    
    # Full matrix
    n_total = n_ch_p1 + n_ch_p2
    full = np.ones((n_total, n_total))
    full[:n_ch_p1, :n_ch_p1] = within_p1
    full[n_ch_p1:, n_ch_p1:] = within_p2
    full[:n_ch_p1, n_ch_p1:] = between
    full[n_ch_p1:, :n_ch_p1] = between.T
    
    return {
        'within_p1': within_p1,
        'within_p2': within_p2,
        'between': between,
        'full': full
    }

In [ ]:
# Function 10: compute_global_plv_hyperscanning

def compute_global_plv_hyperscanning(
    plv_dict: dict[str, NDArray[np.float64]]
) -> dict[str, float]:
    """
    Compute summary statistics for hyperscanning PLV.
    
    Parameters
    ----------
    plv_dict : dict[str, NDArray[np.float64]]
        Output from compute_plv_hyperscanning.
    
    Returns
    -------
    dict[str, float]
        Summary statistics including mean PLV values.
    """
    within_p1 = plv_dict['within_p1']
    within_p2 = plv_dict['within_p2']
    between = plv_dict['between']
    
    # Get upper triangle (excluding diagonal) for within matrices
    mask_p1 = np.triu(np.ones_like(within_p1, dtype=bool), k=1)
    mask_p2 = np.triu(np.ones_like(within_p2, dtype=bool), k=1)
    
    mean_within_p1 = np.mean(within_p1[mask_p1])
    mean_within_p2 = np.mean(within_p2[mask_p2])
    mean_between = np.mean(between)
    
    mean_within = (mean_within_p1 + mean_within_p2) / 2
    ratio = mean_between / mean_within if mean_within > 0 else 0
    
    return {
        'mean_within_p1': float(mean_within_p1),
        'mean_within_p2': float(mean_within_p2),
        'mean_between': float(mean_between),
        'ratio_between_within': float(ratio)
    }

In [ ]:
# Visualization 12: Hyperscanning PLV matrix

# Generate synthetic hyperscanning data
np.random.seed(42)
fs = 500
n_samples = 10000
n_ch_p1 = 4
n_ch_p2 = 4
t = np.arange(n_samples) / fs
freq = 10

# Create data with inter-brain synchrony for specific channel pairs
data_p1 = np.zeros((n_ch_p1, n_samples))
data_p2 = np.zeros((n_ch_p2, n_samples))

# Shared component for some inter-brain synchrony
shared_signal = np.sin(2 * np.pi * freq * t)

# P1 channels
for i in range(n_ch_p1):
    # Within-brain coherence
    data_p1[i] = np.sin(2 * np.pi * freq * t + np.random.randn() * 0.2) 
    data_p1[i] += 0.3 * np.random.randn(n_samples)

# P2 channels - add shared component with P1 for some channels
for i in range(n_ch_p2):
    if i < 2:  # Channels 0-1 have inter-brain synchrony
        data_p2[i] = 0.5 * shared_signal + 0.5 * np.sin(2 * np.pi * freq * t + np.random.randn() * 0.3)
    else:
        data_p2[i] = np.sin(2 * np.pi * freq * t + np.pi + np.random.randn() * 0.5)
    data_p2[i] += 0.3 * np.random.randn(n_samples)

# Compute hyperscanning PLV
band = (8, 12)
plv_hyper = compute_plv_hyperscanning(data_p1, data_p2, fs, band)
stats = compute_global_plv_hyperscanning(plv_hyper)

# Plot full matrix
fig, ax = plt.subplots(figsize=(10, 8))

full_matrix = plv_hyper['full']
im = ax.imshow(full_matrix, cmap='viridis', vmin=0, vmax=1, aspect='equal')
plt.colorbar(im, ax=ax, label='PLV')

# Draw block boundaries
ax.axhline(y=n_ch_p1 - 0.5, color='white', linewidth=3)
ax.axvline(x=n_ch_p1 - 0.5, color='white', linewidth=3)

# Labels
labels = [f'P1-{i+1}' for i in range(n_ch_p1)] + [f'P2-{i+1}' for i in range(n_ch_p2)]
ax.set_xticks(range(len(labels)))
ax.set_yticks(range(len(labels)))
ax.set_xticklabels(labels, rotation=45)
ax.set_yticklabels(labels)

# Block annotations
ax.text(1.5, 1.5, 'Within P1', ha='center', va='center', fontsize=10, 
        color='white', fontweight='bold')
ax.text(5.5, 5.5, 'Within P2', ha='center', va='center', fontsize=10, 
        color='white', fontweight='bold')
ax.text(5.5, 1.5, 'Between\n(P1→P2)', ha='center', va='center', fontsize=10, 
        color='white', fontweight='bold')

ax.set_title(f'Inter-Brain Phase Locking in Hyperscanning\n'
             f'Between-brain mean PLV = {stats["mean_between"]:.3f}', 
             fontweight='bold', fontsize=12)

plt.tight_layout()
plt.show()

print(f"\nSummary Statistics:")
print(f"  Mean within-P1 PLV: {stats['mean_within_p1']:.3f}")
print(f"  Mean within-P2 PLV: {stats['mean_within_p2']:.3f}")
print(f"  Mean between-brain PLV: {stats['mean_between']:.3f}")
print(f"  Between/Within ratio: {stats['ratio_between_within']:.3f}")

---

## Section 9: Statistical Significance for PLV

Even completely random signals produce non-zero PLV due to finite sample size. The theoretical expectation for independent signals is approximately PLV ~ 1/√N where N is the number of samples.

**Surrogate testing**:
1. Shuffle one signal's phase (preserving spectrum)
2. Compute surrogate PLV
3. Repeat many times → null distribution
4. Compare observed PLV to this distribution

**Rayleigh test**: Tests whether phase differences are uniformly distributed (low PLV) or clustered (high PLV).

In [ ]:
# Function 11: plv_surrogate_test

def plv_surrogate_test(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    n_surrogates: int = 500,
    seed: int | None = None
) -> dict[str, Any]:
    """
    Test PLV significance using surrogate data.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal.
    y : NDArray[np.float64]
        Second signal.
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    n_surrogates : int, optional
        Number of surrogate datasets (default: 500).
    seed : int | None, optional
        Random seed for reproducibility.
    
    Returns
    -------
    dict[str, Any]
        Results including observed PLV, null distribution stats, and p-value.
    """
    if seed is not None:
        np.random.seed(seed)
    
    # Compute observed PLV
    plv_observed = compute_plv(x, y, fs, band)
    
    # Generate surrogates by phase shuffling
    nyq = fs / 2
    b, a = signal.butter(4, [band[0]/nyq, band[1]/nyq], btype='band')
    y_filt = signal.filtfilt(b, a, y)
    
    surrogate_plvs = []
    for _ in range(n_surrogates):
        # Phase shuffle y while preserving spectrum
        y_fft = np.fft.fft(y_filt)
        random_phases = np.exp(1j * np.random.uniform(0, 2*np.pi, len(y_fft)))
        # Make symmetric for real output
        random_phases[len(random_phases)//2:] = np.conj(random_phases[1:len(random_phases)//2+1][::-1])
        y_surrogate = np.real(np.fft.ifft(np.abs(y_fft) * random_phases))
        
        plv_surr = compute_plv(x, y_surrogate, fs, band)
        surrogate_plvs.append(plv_surr)
    
    surrogate_plvs = np.array(surrogate_plvs)
    
    # Compute statistics
    null_mean = np.mean(surrogate_plvs)
    null_std = np.std(surrogate_plvs)
    pvalue = np.mean(surrogate_plvs >= plv_observed)
    threshold_95 = np.percentile(surrogate_plvs, 95)
    
    return {
        'plv_observed': float(plv_observed),
        'null_mean': float(null_mean),
        'null_std': float(null_std),
        'pvalue': float(pvalue),
        'threshold_95': float(threshold_95),
        'surrogate_distribution': surrogate_plvs
    }

In [ ]:
# Function 12: rayleigh_test

def rayleigh_test(
    phase_diff: NDArray[np.float64]
) -> dict[str, float]:
    """
    Rayleigh test for non-uniformity of phase differences.
    
    Parameters
    ----------
    phase_diff : NDArray[np.float64]
        Phase difference time series (radians).
    
    Returns
    -------
    dict[str, float]
        Test results with z-statistic and p-value.
    
    Notes
    -----
    High z / low p indicates significantly non-uniform distribution,
    suggesting phase locking.
    """
    n = len(phase_diff)
    
    # Compute resultant length (this is PLV)
    R = np.abs(np.mean(np.exp(1j * phase_diff)))
    
    # Rayleigh's z statistic
    z = n * R**2
    
    # P-value (approximation for large n)
    pvalue = np.exp(-z) * (1 + (2*z - z**2)/(4*n) - (24*z - 132*z**2 + 76*z**3 - 9*z**4)/(288*n**2))
    pvalue = max(0, min(1, pvalue))  # Clip to [0, 1]
    
    return {
        'z_statistic': float(z),
        'pvalue': float(pvalue)
    }

In [ ]:
# Visualization 14: Significance testing

# Generate phase-locked signals
np.random.seed(42)
x, y = generate_phase_locked_signals(10000, 500, 10, plv_target=0.4, seed=42)

# Run surrogate test
results = plv_surrogate_test(x, y, fs=500, band=(8, 12), n_surrogates=500, seed=42)

# Plot
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(results['surrogate_distribution'], bins=30, color=COLORS['signal_1'], 
        alpha=0.7, density=True, label='Null distribution')
ax.axvline(x=results['plv_observed'], color=COLORS['highlight'], linewidth=3, 
           linestyle='-', label=f'Observed PLV = {results["plv_observed"]:.3f}')
ax.axvline(x=results['threshold_95'], color=COLORS['signal_2'], linewidth=2, 
           linestyle='--', label=f'95% threshold = {results["threshold_95"]:.3f}')

ax.set_xlabel('PLV', fontsize=12)
ax.set_ylabel('Density', fontsize=12)
ax.set_title(f'PLV Significance Testing (p = {results["pvalue"]:.4f})', 
             fontsize=14, fontweight='bold')
ax.legend()

# Add significance annotation
if results['pvalue'] < 0.05:
    ax.text(0.95, 0.95, '✓ Significant (p < 0.05)', transform=ax.transAxes,
            ha='right', va='top', fontsize=12, color='green', fontweight='bold')
else:
    ax.text(0.95, 0.95, '✗ Not significant', transform=ax.transAxes,
            ha='right', va='top', fontsize=12, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

---

## Section 10: Comparison with Coherence

PLV and coherence are both frequency-specific connectivity measures, but they differ in important ways:

| Aspect | PLV | Coherence |
|--------|-----|------------|
| What it measures | Pure phase consistency | Phase + amplitude coupling |
| Amplitude sensitivity | None (unit vectors) | Yes (affected by power) |
| Range | 0-1 | 0-1 (magnitude) |
| Volume conduction | Sensitive | Sensitive |

**When to use which**:
- **PLV**: When you specifically care about phase synchrony, independent of amplitude
- **Coherence**: When overall coupling matters, including amplitude covariation
- Often, compute both!

In [ ]:
# Function 13: compare_plv_coherence

def compare_plv_coherence(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, float]:
    """
    Compare PLV and coherence for a signal pair.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal.
    y : NDArray[np.float64]
        Second signal.
    fs : float
        Sampling frequency in Hz.
    band : tuple[float, float]
        Frequency band (low, high) in Hz.
    
    Returns
    -------
    dict[str, float]
        PLV, coherence, and their difference.
    """
    # Compute PLV
    plv = compute_plv(x, y, fs, band)
    
    # Compute coherence using scipy
    f, Cxy = signal.coherence(x, y, fs=fs, nperseg=int(fs))
    
    # Get average coherence in band
    band_mask = (f >= band[0]) & (f <= band[1])
    coherence = np.mean(Cxy[band_mask])
    
    return {
        'plv': float(plv),
        'coherence': float(coherence),
        'difference': float(plv - coherence)
    }

In [ ]:
# Visualization 15: PLV vs Coherence scatter

np.random.seed(42)
n_pairs = 50
fs = 500
n_samples = 5000
band = (8, 12)

plv_values = []
coh_values = []

for i in range(n_pairs):
    # Generate signal pairs with varying coupling
    plv_target = np.random.uniform(0.1, 0.9)
    x, y = generate_phase_locked_signals(n_samples, fs, 10, plv_target=plv_target, seed=i)
    
    # Add amplitude variations
    amp_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.5 * np.arange(n_samples) / fs)
    x = x * amp_mod
    
    results = compare_plv_coherence(x, y, fs, band)
    plv_values.append(results['plv'])
    coh_values.append(results['coherence'])

# Plot
plt.figure(figsize=(8, 8))
plt.scatter(coh_values, plv_values, c=COLORS['signal_1'], alpha=0.6, s=50)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Identity line')
plt.xlabel('Coherence', fontsize=12)
plt.ylabel('PLV', fontsize=12)
plt.title('PLV vs Coherence Comparison\n(Usually correlated but not identical)', 
          fontsize=14, fontweight='bold')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend()
plt.grid(True, alpha=0.3)

# Add correlation
corr, _ = pearsonr(plv_values, coh_values)
plt.text(0.05, 0.95, f'r = {corr:.3f}', transform=plt.gca().transAxes,
         fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

---

## Section 12: Hands-On Exercises

Now it's your turn! Complete the following exercises to reinforce your understanding of PLV.

In [ ]:
# Exercise 1: Basic PLV
# Generate two 10 Hz signals with a known phase offset (e.g., π/3)
# Compute PLV and verify it's approximately 1 (regardless of offset value)
# Then add phase noise and observe PLV decrease

# YOUR CODE HERE
# x, y = generate_phase_locked_signals(...)
# plv = compute_plv(...)
# print(f"PLV with constant offset: {plv:.3f}")

In [ ]:
# Exercise 2: Unit Circle Visualization
# Generate phase-locked signals, extract phases, compute differences
# Plot phase differences on unit circle
# Verify: clustered for high PLV, spread for low PLV

# YOUR CODE HERE

In [ ]:
# Exercise 3: Volume Conduction Demonstration
# Simulate volume conduction (same signal at two "electrodes" + noise)
# Compute PLV → should be high
# Note: this is SPURIOUS! Save results for comparison with PLI in G02

# YOUR CODE HERE

In [ ]:
# Exercise 4: Time-Resolved PLV
# Create signals where PLV changes over time
# First half: phase locked (PLV high)
# Second half: not locked (PLV low)
# Compute sliding window PLV and verify you capture the transition

# YOUR CODE HERE

In [ ]:
# Exercise 5: PLV Matrix
# Create 8-channel simulated data with cluster structure:
# - Channels 1-4 phase locked within cluster
# - Channels 5-8 phase locked within cluster
# - Between clusters: weak coupling
# Compute PLV matrix and verify it reveals your designed structure

# YOUR CODE HERE

In [ ]:
# Exercise 6: Hyperscanning PLV
# Create two-participant data (6 channels each)
# Add between-brain phase locking for specific channel pairs
# Compute hyperscanning PLV and identify the synchronized pairs
# Test against surrogates for significance

# YOUR CODE HERE

In [ ]:
# Exercise 7: PLV vs Coherence
# Create a scenario where PLV and coherence differ:
# High amplitude correlation but variable phase
# Compute both metrics and discuss which better captures your scenario

# YOUR CODE HERE

---

## Summary

### Key Takeaways

1. **PLV measures consistency of phase difference over time**
   - Formula: PLV = |mean(e^(i×Δφ))|
   - Range: 0 (no locking) to 1 (perfect locking)

2. **PLV measures phase consistency, not actual phase value**
   - 90° offset with perfect consistency → PLV = 1

3. **Computation pipeline**:
   - Filter → Hilbert → phases → difference → unit vectors → average → magnitude

4. **⚠️ Sensitive to volume conduction** (like coherence)
   - Use PLI/wPLI if within-brain analysis is a concern

5. **Time-resolved PLV**: Sliding window approach for dynamic connectivity

6. **PLV matrix**: Symmetric, diagonal = 1, all values 0-1

7. **For hyperscanning**: Most common metric for inter-brain synchrony
   - Inter-brain PLV is safe from volume conduction (separate heads!)

8. **Statistical testing**: Surrogates or Rayleigh test

9. **PLV ignores amplitude** (unlike coherence)

10. **Next**: PLI addresses the volume conduction problem (G02)

---

## Discussion Questions

1. Two colleagues claim PLV = 0.6 between frontal and occipital electrodes. What questions would you ask before believing this reflects true connectivity?

2. You're studying conversation and find high inter-brain PLV at the speech rate (4-5 Hz theta). Is this "neural synchrony" or just both people responding to the same acoustic rhythm? How would you investigate?

3. PLV = 0.3. Is this "low" or "moderate"? What additional information would you need to interpret this value?

4. Your time-resolved PLV shows spikes exactly when both participants press buttons (behavioral synchrony task). Is this neural PLV or movement artifact? How would you tell?

5. A reviewer asks "Why PLV instead of coherence?" for your hyperscanning study. How do you justify your metric choice?